In [4]:
import numpy as np
import pandas as pd
import yfinance as yf
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

# Step 1: Loading historical stock data using Yahoo Finance
symbol = "AAPL"  # using apple as an example
start_date = "2020-01-01"
end_date = "2022-01-01"
data = yf.download(symbol, start=start_date, end=end_date)



[*********************100%%**********************]  1 of 1 completed


In [5]:
# Step 2: Preprocess data
data['Returns'] = data['Adj Close'].pct_change()
data.dropna(inplace=True)



In [6]:
# Step 3: Define features and target variable
X = data[['Open', 'High', 'Low', 'Close', 'Volume']].values
y = data['Returns'].values



In [7]:
# Step 4: Normalize data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.reshape(-1, 1))



In [8]:
# Step 5: Split data into training and testing sets
split_ratio = 0.8
split_index = int(split_ratio * len(X_scaled))
X_train, X_test = X_scaled[:split_index], X_scaled[split_index:]
y_train, y_test = y_scaled[:split_index], y_scaled[split_index:]



In [10]:
# Step 6: Define a simple LSTM model using PyTorch
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        #print("LSTM Output Shape:", out.shape)
        out = self.fc(out[:, -1, :])  # Use the output of the last time step
        return out

input_size = 5
hidden_size = 50
num_layers = 2
output_size = 1

model = LSTMModel(input_size, hidden_size, num_layers, output_size)



In [11]:
# Step 7: Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



In [12]:
# Step 8: Train the model
num_epochs = 10
batch_size = 64

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

for epoch in range(num_epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        end_idx = min(i + batch_size, len(X_train_tensor))
        inputs = X_train_tensor[i:end_idx]
        targets = y_train_tensor[i:end_idx]

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()



LSTM Output Shape: torch.Size([64, 50])


IndexError: too many indices for tensor of dimension 2

In [ ]:
# Step 9: Make predictions using the trained model
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_pred_scaled = model(X_test_tensor).detach().numpy()
y_pred = scaler.inverse_transform(y_pred_scaled)



In [ ]:
# Step 10: Create trading signals based on predictions
data['Signal'] = 0
data.iloc[split_index:, data.columns.get_loc('Signal')] = np.where(y_pred > 0, 1, -1)



In [ ]:
# Step 11: Backtesting and performance evaluation
data['Strategy'] = data['Signal'] * data['Returns']
cumulative_strategy_returns = (data['Strategy'] + 1).cumprod()



In [ ]:
# Step 12: Visualize results
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(data.index, cumulative_strategy_returns, label='Strategy Returns')
plt.plot(data.index, (data['Returns'] + 1).cumprod(), label='Buy and Hold Returns')
plt.legend()
plt.title('LSTM Trading Strategy vs Buy and Hold')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.show()